In [1]:
import sys
import os
cwd = os.getcwd() + '\\main\\'
sys.path.append(cwd)
print(cwd)

C:\Users\Deep-Learning-PC\Desktop\Various_for_biomedical_semantics_journal\Rebuttal\deep-align\main\


In [2]:
import pickle
from time import time
from sklearn.metrics.pairwise import cosine_distances
from main import siamese_word_model_neg_sampling_distillation_loss
from main import ppdb_utils
from main import utils
from main import params
from main.tree import addOOVwords
from main import evaluate
from main import marriage
import numpy as np
import lasagne
import theano
import theano.tensor as T

1 #define _CUDA_NDARRAY_C
2 
3 #include <Python.h>
4 #include <structmember.h>
5 #include "theano_mod_helper.h"
6 
7 #include <numpy/arrayobject.h>
8 #include <iostream>
9 
10 #include "cuda_ndarray.cuh"
11 
12 #ifndef CNMEM_DLLEXPORT
13 #define CNMEM_DLLEXPORT
14 #endif
15 
16 #include "cnmem.h"
17 #include "cnmem.cpp"
18 
19 //If true, when there is a gpu malloc or free error, we print the size of allocated memory on the device.
20 #define COMPUTE_GPU_MEM_USED 0
21 
22 //If true, we fill with NAN allocated device memory.
23 #define ALLOC_MEMSET 0
24 
25 //If true, we print out when we free a device pointer, uninitialize a
26 //CudaNdarray, or allocate a device pointer
27 #define PRINT_FREE_MALLOC 0
28 
29 //If true, we do error checking at the start of functions, to make sure there
30 //is not a pre-existing error when the function is called.
31 //You probably need to set the environment variable
32 //CUDA_LAUNCH_BLOCKING=1, and/or modify the CNDA_THREAD_SYNC
33 //preprocessor macro 

856         Py_DECREF(self_sum);
857         PyErr_SetString(PyExc_MemoryError, "failed to alloc internal storage");
858         return NULL;
859     }
860     for (int i = 0; i < len; ++i)
861     {
862         PyObject *o_i = PySequence_GetItem(py_reduce_mask, i);
863         int o_i_int = PyInt_AsLong(o_i);
864         Py_XDECREF(o_i);
865         if (PyErr_Occurred())
866         {
867             Py_DECREF(self_sum);
868             free(dimshuffle_pattern);
869             return NULL;
870         }
871         if (o_i_int) // this is a dimension over which we are reducing
872         {
873             sum_dims[i] = 1;
874         }
875         else
876         {
877             sum_dims[i] = CudaNdarray_HOST_DIMS(self)[i];
878             dimshuffle_pattern[n_remaining_dims++] = i;
879         }
880     }
881     if (0   || CudaNdarray_alloc_contiguous(self_sum, len, sum_dims)
882             || CudaNdarray_reduce_sum(self_sum, self)
883             || CudaNdarray_dimshuffle(sel

2119 
2120 /*
2121  * We need this inplace div for cuda/tests/test_basic_ops.py:test_shared_options
2122  * It returns py_self on success with an additional reference. Else NULL.
2123  */
2124 // Will be called by __idiv__ in Python
2125 static PyObject *
2126 CudaNdarray_inplace_div(PyObject* py_self, PyObject * py_other)
2127 {
2128     if (CudaNdarray_inplace_elemwise(py_self, py_other, IDIV))
2129     {
2130         return NULL;
2131     }
2132     Py_INCREF(py_self);
2133     return py_self;
2134 }
2135 
2136 // The PyNumberMethods struct layout changed in a non-trivial way from 2 to 3.
2137 #if PY_MAJOR_VERSION == 3
2138 static PyNumberMethods CudaNdarrayNumberMethods =
2139 {
2140     (binaryfunc)CudaNdarray_add,  //binaryfunc nb_add;  __add__
2141     0,  //binaryfunc nb_subtract;
2142     0,  //binaryfunc nb_multiply;
2143     0,  //binaryfunc nb_remainder;
2144     0,  //binaryfunc nb_divmod;
2145     0,  //ternaryfunc nb_power;
2146     0,  //unaryfunc nb_negative;
2147     

3396         goto CudaNdarray_dot_fail;
3397     }
3398     if (((CudaNdarray*)l)->nd != 2)
3399     {
3400         PyErr_SetString(PyExc_TypeError, "need 2d CudaNdarray arg for now");
3401         goto CudaNdarray_dot_fail;
3402     }
3403     if (((CudaNdarray*)r)->nd != 2)
3404     {
3405         PyErr_SetString(PyExc_TypeError, "need 2d CudaNdarray arg for now");
3406         goto CudaNdarray_dot_fail;
3407     }
3408     rval = CudaNdarray_New();
3409     if (!rval)
3410     {
3411         goto CudaNdarray_dot_fail;
3412     }
3413     int dims[2];
3414     dims[0] = CudaNdarray_HOST_DIMS((CudaNdarray*)l)[0];
3415     dims[1] = CudaNdarray_HOST_DIMS((CudaNdarray*)r)[1];
3416     if (CudaNdarray_alloc_contiguous((CudaNdarray*)rval, 2, dims))
3417     {
3418         goto CudaNdarray_dot_fail;
3419     }
3420     if (CudaNdarray_gemm(1.0, (CudaNdarray*)l, (CudaNdarray*)r, 0.0, (CudaNdarray*)rval))
3421     {
3422         goto CudaNdarray_dot_fail;
3423     }
3424 
3425     return rva

4634                     }
4635                     const int stride0 = a_str[reduce_dim0];
4636                     const int stride1 = a_str[rd];
4637                     for (int ii = 0; ii < dims_a[rd]; ++ii)
4638                     {
4639                         const float * a_data_ri = a_data_i + ii * stride1;
4640                         const float * a_data_ri_max = a_data_ri + dims_a[reduce_dim0] * stride0;
4641                         while (a_data_ri != a_data_ri_max)
4642                         {
4643                             sum += a_data_ri[0];
4644                             a_data_ri += stride0;
4645                         }
4646                     }
4647                 };
4648                 break;
4649             default:
4650                 {
4651                     for (unsigned int reduce_i = 0; reduce_i < n_reduce_elements; ++reduce_i)
4652                     {
4653                         //TODO: optimize this loop to work more like theano's Elemwi

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc fatal   : Cannot find compiler 'cl.exe' in PATH

['nvcc', '-shared', '-O3', '-Xlinker', '/DEBUG', '-D HAVE_ROUND', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=m18715462c72ed6afcd7ca5d52813ce90,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,/Zi,/MD', '-IC:\\Users\\Deep-Learning-PC\\Anaconda3\\envs\\tensorflow\\lib\\site-packages\\theano\\sandbox\\cuda', '-IC:\\Users\\Deep-Learning-PC\\Anaconda3\\envs\\tensorflow\\lib\\site-packages\\numpy\\core\\include', '-IC:\\Users\\Deep-Learning-PC\\Anaconda3\\envs\\tensorflow\\include', '-IC:\\Users\\Deep-Learning-PC\\Anaconda3\\envs\\tensorflow\\lib\\site-packages\\theano\\gof', '-o', 'C:\\Users\\Deep-Learning-PC\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.16299-SP0-Intel64_Family_6_Model_79_Stepping_1_GenuineIntel-3.5.3-64\\cuda_ndarray\\cuda_ndarray.pyd', 'mod.cu

Using TensorFlow backend.


###### Load the Configuration Parameters:

For the Out-of-Vocabulary (OOV) words, we initialize them with values sampled from a normal distribution ($X \sim \mathcal{N}(\mu=0,\,\sigma^{2}=0.01)\,$).

Please set the input of the load_from_yaml method to:
1. 'data/ma-nci/ma-nci_params.yaml' for performing ontology matching between the **Adult Mouse Anatomical Dictionary** and the **Foundation Model of Anatomy** ontology.
2. 'data/fma-nci/fma-nci_params.yaml' for performing ontology matching between the **Foundation Model of Anatomy** ontology and the **NCI Thesaurus**.
3. 'data/fma-snomed/fma-snomed_params.yaml' for performing ontology matching between the **Foundation Model of Anatomy** ontology and **SNOMED CT**.

In [3]:
params = params.params()
params.load_from_yaml('data/fma-snomed/fma-snomed_params.yaml')
(words, We) = utils.getWordmap(params.wordfile)
print('The number of words that existed in the pre-trained words vectors trained on PubMed and PMC is: %d' % (len(words)))
# Read the training data and add the OOV words in the dictionary.
examples = utils.getDataset(params.train, words)
We = addOOVwords(examples, words, We, mean=0, sigma=0.01)

params.batchsize = len(examples)

The number of words that existed in the pre-trained words vectors trained on PubMed and PMC is: 3418


###### Print the resulted Configuration Parameters:

In [ ]:
print(params)

###### Initialize the Phrase Retrofitting Component:

In [ ]:
model = siamese_word_model_neg_sampling_distillation_loss.ppdb_word_model(We, params)

###### Extract possible examples of 'descriptive associated' terms:

In [ ]:
terms_of_ontology_1 = params.terms_of_ontology_1
terms_of_ontology_2 = params.terms_of_ontology_2
ontology_1_ants = utils.getAntonyms(terms_of_ontology_1, words)
ontology_2_ants = utils.getAntonyms(terms_of_ontology_2, words)
ants = ontology_1_ants + ontology_2_ants
utils.getAntRepresentations(model, ants)
syns = utils.createSet(examples, words)
utils.getAntRepresentations(model, syns)

###### Train the Phrase Retroffiting Component:

In [ ]:
ppdb_utils.train(model, examples, words, params, synonyms=syns, antonyms=ants, start=2)

###### Run (1) the <cite>[McVitie et al.][1]</cite> algorithm for solving the Stable Marriage Assignment problem and (2) display the resulted performance:


[1]:https://link.springer.com/article/10.1007/BF01934199

In [ ]:
from time import time
start_time = time()
alignments = marriage.ontology_alignment(model, terms_of_ontology_1, terms_of_ontology_2, words, ceil=0.2)
end_time = time()
print("Total matching time:", (end_time - start_time))
results = marriage.alignment_evaluation(model, words, alignments, params.ground_truth_alignments)

###### Preprocess the synonymy data so as to feed them to the <cite>[Denoising Autoencoder][1]</cite>

Note: We preprocess the data as described in <cite>[LeCun et al.][2]</cite>


[1]:https://dl.acm.org/citation.cfm?id=1390294
[2]:http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf

In [ ]:
from main.utils import prepare_data

left = []
for example in examples:
    example[0].populate_embeddings(words)
    left.append(example[0].embeddings)
right = []
for example in examples:
    example[1].populate_embeddings(words)
    right.append(example[1].embeddings)
    
X1, M1 = prepare_data(left)
X2, M2 = prepare_data(right)
# Data preprocessing
embg1 = model.feedforward_function(X1, M1)
embg2 = model.feedforward_function(X2, M2)
embg1 = embg1 - embg1.mean(axis=1, keepdims=True)
embg2 = embg2 - embg2.mean(axis=1, keepdims=True)

embg1 = embg1/ np.linalg.norm(embg1)
embg2 = embg1/ np.linalg.norm(embg2)
# End of Data preprocessing

# As the number of training data is not huge, we extract only the 1% of them for checking the validation loss.
train_len = int(len(embg1)*0.99)
train_origins = embg1[:train_len]
train_targets = embg2[:train_len]
test_origins = embg1[train_len:]
test_targets = embg2[train_len:]

train_all1 = np.concatenate((train_origins, train_origins, train_targets, train_targets), axis=0)
train_all2 = np.concatenate((train_origins, train_targets, train_origins, train_targets), axis=0)

###### Initialize the Denoising Autoencoder (DAE):

In [ ]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 6.25, assuming the input is 200 floats

# this is our input placeholder
input_img = Input(shape=(200,))
# "encoded" is the encoded representation of the input
v = 0.4
dropped_input = Dropout(v)(input_img)

encoded = Dense(encoding_dim, 
                activation='relu',
                activity_regularizer=regularizers.l1(10e-05)
               )(dropped_input)
decoded = Dense(200, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

###### Define the DAE's loss and the optimization method:

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

###### Train the DAE:

In [ ]:
time_callback = evaluate.TimeHistory()
autoencoder.fit(train_all1, train_all2,
                epochs=15,
                batch_size=256,
                shuffle=True,
                validation_data=(test_origins, test_targets),
                callbacks=[time_callback])
print("DAE's total training time:", (time_callback.overall_training_time()))

###### Define some helpful functions so as to get the output of the DAE:

In [ ]:
def sen2em(model,sent,words):
    d = evaluate.sen2Embgs(model,sent,words)
    emb = np.zeros(200)
    
    for key, value in d.items():
        emb += value
    return emb/len(d)

def ae_sim(model, string, words):
    from sklearn.metrics.pairwise import cosine_similarity
    sentence_1, sentence_2 = string.split(', ')
    x1 = sen2em(model,sentence_1,words)
    x2 = sen2em(model,sentence_2,words)
    encoded_imgs_1 = encoder.predict(x1.reshape(1,200))
    encoded_imgs_2 = encoder.predict(x2.reshape(1,200))
    return np.squeeze(cosine_similarity(encoded_imgs_1,encoded_imgs_2))

###### Run the Outlier Detection Component based on the DAE:

In [ ]:
cnt=0
detected_outliers=[]
outlier_threshold = 0.2
for x in results[0]:
    string = x.split(') -> ')[0][1:]
    ae_value = 1-ae_sim(model, string, words)
    if ae_value >= outlier_threshold:
        detected_outliers.append(x)
        cnt+=1
    elif ae_value < outlier_threshold:
        print(x + ' ||| ' +str(ae_value))
print('The number of detected outliers is: %d' % (cnt))

###### Compute the number of correctly detected misalignments and the number of correct alignments that were marked wrongly as misalignments:

In [ ]:
wrong, correct = 0, 0
for x in detected_outliers:
    if '\x1b[0m' in x:
        wrong+=1
    else:
        correct+=1

######  Display the resulted performance after the application of the DAE based Outlier Detection Component:

In [ ]:
print('The DAE outlier detector discovered correctly %d misalignments' % (wrong))
print('However, the DAE outlier detector also confused %d true alignments as misalignments' % (correct))

precision = (1.0*(results[3]-correct))/((results[3]-correct)+(results[4]-wrong))
recall = (1.0*(results[3]-correct))/results[2]

print('The new precision is: %f' % (precision))
print('The new recall    is: %f' % (recall))
print('The new F1-score  is  %f' % ((2.0*precision*recall)/(precision+recall)))